In [ ]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 588.3 MB 17 kB/s 
     |████████████████████████████████| 439 kB 57.0 MB/s 
     |████████████████████████████████| 6.0 MB 37.9 MB/s 
     |████████████████████████████████| 1.7 MB 66.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/IMDB Dataset.csv')

df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
df['is_positive'] = (df['sentiment'] == 'positive').astype(int)

In [ ]:
df.drop(columns=['sentiment'], inplace=True)

In [ ]:
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df

,DATA_COLUMN,LABEL_COLUMN
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [ ]:
df.head(5)

,DATA_COLUMN,LABEL_COLUMN
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
df['LABEL_COLUMN'].value_counts()

1    25000
0    25000
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]

In [ ]:
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
df_negative

,DATA_COLUMN,LABEL_COLUMN
3,Basically there's a family where a little boy ...,0
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
10,Phil the Alien is one of those quirky films wh...,0
11,I saw this movie when I was about 12 when it c...,0
...,...,...
49994,This is your typical junk comedy.<br /><br />T...,0
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [ ]:
# Для тестовой выборки берем последние 10% негативных отзывов и последние 10% позитивных отзывов

In [ ]:
n_test = df_negative.shape[0] // 10
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 10
df_positive_test = df_positive.tail(n_test)

In [ ]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
44910,One of the most heart-warming foreign films I'...,1
44911,David Lynch's (1999) film of John Roach / Mary...,1
44916,"This is definitely Nolan's most intimite,and t...",1
44925,Before I watched this tv movie I did not know ...,1
44927,"I just managed to find a copy of ""Mission Cleo...",1
...,...,...
49983,"I loved it, having been a fan of the original ...",1
49985,Imaginary Heroes is clearly the best film of t...,1
49989,I got this one a few weeks ago and love it! It...,1
49992,John Garfield plays a Marine who is blinded by...,1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test.sample(10)

,DATA_COLUMN,LABEL_COLUMN
49862,I saw this series when it world premiered at t...,1
47767,I ran across Yvette McClendon in a film at the...,0
45910,I liked the movie a real lot. Wanted to see it...,1
46529,The humor implicit in the complete title prove...,1
46762,I have seen this movie 4 times in 5 months and...,1
49647,I loved that this film recognizes the intellig...,1
49766,This might be for those who have been to summe...,0
46108,I remember being terrified of movie blood when...,1
49940,<br /><br />One would expect a movie with a fa...,0
45972,I was unlucky enough to have seen this at the ...,0


In [ ]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2500
1    2500
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Для обучающей выборки берем первые 2.5% из начала датасета.

In [ ]:
n_train = df_negative.shape[0] // 40
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 40
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
df_balanced_train['LABEL_COLUMN'].value_counts()

0    625
1    625
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_balanced_train.sample(10)

,DATA_COLUMN,LABEL_COLUMN
141,Ye Lou's film Purple Butterfly pits a secret o...,0
23,"First of all, let's get a few things straight ...",0
21,I had the terrible misfortune of having to vie...,0
28,This movie was so frustrating. Everything seem...,0
646,It wasn't until I looked at the trivia section...,0
1146,I am sad to say that I disagree with other peo...,1
184,"A very ordinary made-for-tv product, ""Tyson"" a...",0
1087,Maria Braun is an extraordinary woman presente...,1
1115,"In theory, films should be a form of entertain...",0
283,I am not so old that I can't remember laughing...,0


In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")


In [ ]:
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
history = model.fit(X_train, y_train, epochs=10)

Epoch 1/10
40/40 [==============================] - 589s 14s/step - loss: 0.7081 - accuracy: 0.5160 - precision: 0.5155 - recall: 0.5312
Epoch 2/10
40/40 [==============================] - 561s 14s/step - loss: 0.6770 - accuracy: 0.5848 - precision: 0.5989 - recall: 0.5136
Epoch 3/10
40/40 [==============================] - 558s 14s/step - loss: 0.6753 - accuracy: 0.5872 - precision: 0.6071 - recall: 0.4944
Epoch 4/10
40/40 [==============================] - 558s 14s/step - loss: 0.6809 - accuracy: 0.5656 - precision: 0.5557 - recall: 0.6544
Epoch 5/10
40/40 [==============================] - 558s 14s/step - loss: 0.6561 - accuracy: 0.6144 - precision: 0.6367 - recall: 0.5328
Epoch 6/10
40/40 [==============================] - 556s 14s/step - loss: 0.6404 - accuracy: 0.6288 - precision: 0.6260 - recall: 0.6400
Epoch 7/10
40/40 [==============================] - 561s 14s/step - loss: 0.6291 - accuracy: 0.6392 - precision: 0.6422 - recall: 0.6288
Epoch 8/10
40/40 [=======================

In [ ]:
y_predicted = model.predict(X_test)

157/157 [==============================] - 2230s 14s/step


In [ ]:
y_predicted = y_predicted.flatten()
y_predicted


array([0.2712389 , 0.4355559 , 0.38521293, ..., 0.54855436, 0.67136115,
       0.42066377], dtype=float32)

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted


array([0, 0, 0, ..., 1, 1, 0])

In [ ]:
y_test_array = np.array(y_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predicted)

0.6868

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test_array, y_predicted)

0.732801595214357

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test_array, y_predicted)

0.588

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test_array, y_predicted)

0.6524633821571239

In [ ]:
df_results_for_reviews_on_movies = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
df_results_for_reviews_on_movies.loc['reviews on movies', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_movies.loc['reviews on movies', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_movies.loc['reviews on movies', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_movies.loc['reviews on movies', 'f1_score'] =  f1_score(y_test_array, y_predicted)

In [ ]:
df_results_for_reviews_on_movies

,accuracy,precision,recall,f1_score
reviews on movies,0.6868,0.732802,0.588,0.652463


In [ ]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/bert_trained_on_imdb_11_december'

In [ ]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/bert_trained_on_imdb_11_december


In [ ]:
model.save(saved_model_path, include_optimizer=True) 